## Import Necessary Modules

In [177]:
import pandas as pd
import datetime
import ast
import numpy as np
import ast

## Step 1: Analyze Data to Determine Best Model

I will be starting off with the **Users** schema and determining the best way to model it first.

In [2]:
# Since we are working with a line-separated JSON file, it is necessary to include the lines parameter
users = pd.read_json('users.json', lines=True)
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [4]:
# Getting some basic information regarding current data types and how much missing data is present
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


As we can see above, there isn't too much missing data; however, several features are encapsulated within dictionaries. The next steps would be to extract the data from these dictionaries.

In [6]:
# Extract the user id from the dictionary
users['_id'] = [row['$oid'] for row in users['_id']]
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [7]:
# While it wouldn't be an issue to perform list comprehensions to extract the dates from the timestamps,
# it will be less repetitive to create a function and use that on the whole column instead.

def extract_date(row):
    '''
    This function takes in a row from a dataframe
    and extracts the timestamp(in ms) from it and converts 
    it into an acceptable timestamp(in s). It will then
    convert the timestamp to Pacific time to make it easier to
    interpret. If there is no timestamp associated with the row
    then it will return None.
    '''
    try:
        date = datetime.datetime.fromtimestamp(row['$date'] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        return date
    except TypeError:
        return None

In [8]:
users['createdDate'] = users['createdDate'].apply(lambda row: extract_date(row))
users['lastLogin'] = users['lastLogin'].apply(lambda row: extract_date(row))
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 07:25:30,2021-01-03 07:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI


In [10]:
# Renaming _id column to user_id for more clarity
users.rename(columns={'_id':'user_id'}, inplace=True)
users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 07:25:30,2021-01-03 07:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI


In [13]:
# Casting the date columns to datetime objects for more optimized storage and functionality
users['createdDate'] = pd.to_datetime(users['createdDate'])
users['lastLogin'] = pd.to_datetime(users['lastLogin'])

In [16]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       495 non-null    object        
 1   active        495 non-null    bool          
 2   createdDate   495 non-null    datetime64[ns]
 3   lastLogin     433 non-null    datetime64[ns]
 4   role          495 non-null    object        
 5   signUpSource  447 non-null    object        
 6   state         439 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 23.8+ KB


In [13]:
# OPTIONAL
# We still have some missing data within the signupsource and state columns
# One way to tackle it would be to fill it in with some pre-determined values
# which would provide us with a more measurable metric(as done below). Another option
# would be to use the most common (mode) values and use them to fill in the missing
# data. However, this could lead to data quality issues especially in the state column
# as there are a substantial amount of unique states.
users['signUpSource'].fillna('Missing', inplace=True)
users['state'].fillna('Missing', inplace=True)
users.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 07:25:30,2021-01-03 07:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 07:24:04,2021-01-03 07:25:37,consumer,Email,WI


#### Potential Optimizations:
This brings us to the end of cleaning up the Users data. There are several optimizations that could be made before putting it into the warehouse and I will discuss a few here. Firstly, I would strongly suggest converting the **role** column to a numeric one. Since there are only two roles that it captures (consumer and staff), they can be stroed as 0 : staff and 1 : consumer. While this change will not make a substantial difference with this amount of data, but when you scale the data up having a int2 datatype instead of a varchar would save a lot of space. Additionally, a similar thing can be done with the **signUpSource** column. The meaning of these numbers can be very easily stored in a document or within the DDL script where the table is generated.

#### Data Issues:
One issue that I noticed with the data is that about **12.5%** of the lastlogin data is missing. This is not something that should be missing since it should be tracked automatically and there aren;t many external reasons that would stop the collection of this data. A similar case can be made for signUpSource and state; however, those columns contain data that could be contaminated due to VPN usage or other external sources.

#### Feature Engineering Options:
With any data that is collected, there is always potential to make it more useful. One of the ideas that I had was to revamp the active filter that is used. Since we have the account creation date and the last login date, a threshhold can be introduced that would allow for the automation of determining the status of the account as well as even grouping it into categories such as daily users, users that just downloaded it to try it and then stopped using it, and truly inactive users. Additionally, the current active column could be repurposed to a flag for users that have bad intentions (scams, harrassment) which could be where actual employees had to terminate accounts.

### 

Let's move on to the **brands** schema now.

In [22]:
# Similarly, do not forget the lines parameter
brands = pd.read_json('brands.json', lines=True)
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [23]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB


After having a look above, we notice that there is a **lot** of data misiing in a few columns. Let's dig in and see what can be done

In [24]:
# Extract brand_id from dictionary
brands['_id'] = [row['$oid'] for row in brands['_id']]
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [25]:
brands['categoryCode'].unique()

array(['BAKING', 'BEVERAGES', 'CANDY_AND_SWEETS', nan,
       'HEALTHY_AND_WELLNESS', 'GROCERY', 'PERSONAL_CARE',
       'CLEANING_AND_HOME_IMPROVEMENT', 'BEER_WINE_SPIRITS', 'BABY',
       'BREAD_AND_BAKERY', 'OUTDOOR', 'DAIRY_AND_REFRIGERATED',
       'MAGAZINES', 'FROZEN'], dtype=object)

In [26]:
brands['category'].unique()

array(['Baking', 'Beverages', 'Candy & Sweets', 'Condiments & Sauces',
       'Canned Goods & Soups', nan, 'Magazines', 'Breakfast & Cereal',
       'Beer Wine Spirits', 'Health & Wellness', 'Beauty', 'Baby',
       'Frozen', 'Grocery', 'Snacks', 'Household', 'Personal Care',
       'Dairy', 'Cleaning & Home Improvement', 'Deli',
       'Beauty & Personal Care', 'Bread & Bakery', 'Outdoor',
       'Dairy & Refrigerated'], dtype=object)

In the above two lines, we compared the unique values that are present in the category column and the category code column. Now, the category_code column is missing a lot of data as we saw above and it pretty much captures the same data as the category column. Hence, I will choose to eliminate the category_code column as it is not providing any extra information to us.

In [27]:
brands.drop(columns=['categoryCode'], inplace=True)
brands.head()

,_id,barcode,category,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


Now, let's have a look at the topBrand column which is the second culprit.

In [28]:
brands['topBrand'] = brands['topBrand'].replace({1.0 : True, 0.0 : False})
brands.head()

,_id,barcode,category,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,False,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,False,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,False,TEST BRANDCODE @1612366146827


Initially, the topBrand column was using 1s and 0s to indicate whether the brand is a top brand or not. While there is nothing wrong with this, it was stored as a float which would take up way more space than it needs to. A simple solution is to just cast it to a boolean value or even just an int2. Now, to deal with the mising data there is a slight problem. We can't just drop the column like we did earlier as it does provide us with some unique insight; but the fact that more than half the data is missing is still a big issue. Now, the easy thing to do would be to just say that if the data is missing then we just assume that it is not a top brand. This approach would actually work pretty well since most brands are not top brands, but the cost would be that we might miss out on the few that might be. This decision is one that would need to be taken as a team or after some deliberation with a PM, but I will just take it myself for now and replace the missing data with False.

In [33]:
brands['topBrand'] = brands['topBrand'].replace({np.nan : True})
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   _id        1167 non-null   object
 1   barcode    1167 non-null   int64 
 2   category   1012 non-null   object
 3   cpg        1167 non-null   object
 4   name       1167 non-null   object
 5   topBrand   1167 non-null   bool  
 6   brandCode  933 non-null    object
dtypes: bool(1), int64(1), object(5)
memory usage: 56.0+ KB


That looks much better. Now, let's have a look at the cpg column.

In [34]:
brands['cpg'][1]

{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, '$ref': 'Cogs'}

Looks like there's two little bits of information hiding in the dictionaries.

In [35]:
brands['cpg_id'] = [row['$id']['$oid'] for row in brands['cpg']]
brands['cpg_ref'] = [row['$ref'] for row in brands['cpg']]
brands.head()

,_id,barcode,category,cpg,name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,False,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,False,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,False,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs


In [36]:
brands.drop(columns=['cpg'], inplace=True)
brands.head()

,_id,barcode,category,name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,test brand @1612366101024,False,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,Starbucks,False,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs


In [37]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   _id        1167 non-null   object
 1   barcode    1167 non-null   int64 
 2   category   1012 non-null   object
 3   name       1167 non-null   object
 4   topBrand   1167 non-null   bool  
 5   brandCode  933 non-null    object
 6   cpg_id     1167 non-null   object
 7   cpg_ref    1167 non-null   object
dtypes: bool(1), int64(1), object(6)
memory usage: 65.1+ KB


Before I ramble on for a few small paragraphs, let's go over what I just did. Firstly, I extracted the id and reference type from the cpg column and made them into their own columns. Once that was done, there was no longer any need for the cpg column so I dropped it.

In [38]:
# Almost forgot to change the _id to something with more clarity
brands.rename(columns={'_id':'brand_id'}, inplace=True)
brands.head()

,brand_id,barcode,category,name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,test brand @1612366101024,False,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,Starbucks,False,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs


In [44]:
brands['category'].nunique()

23

#### Potential Optimizations
Now that the brands data is taken care of, let's see if there are any ways that could be used to optimize this table when it is being stored. Unlike the users table, most of the columns in brands contain quite a few unique values so transforming them into integers wouldn't work all too well. However, the cpg_ref column can be transformed in this way. 1 for Cogs and 0 for Cpgs and we are saving a decent amount of space. Similarly, we could do the same for the category column as it only has 23 unique values. This is not too many and I've seen up to a 100 unique values being transformed to integers and then the transformation matrix being stored in a different script or common document for reference.

#### Data Issues
The issues this time were a bit too big to save till the end. The amount of data missing in the categoryCode and topBrand columns was a bit too severe and was dealt with above. The only concern that I would look into this point would be that wherever the topBrand data comes from needs to be inspected to ensure that it is working properly.

#### Feature Engineering
Unfortunately, with just the data  in this table it is quite hard to come up with any new features taht could be generated. However, if we do look ahead we could definitely use the users and receipts data to make the topBrand data rely on sales and transactions. We could also incorporate how much each brand is selling and how many unique or returning customers it has.

### 

Time to look at the big one, the **receipts** data.

In [41]:
# Once again, do not forget the lines parameter
receipts = pd.read_json('receipts.json', lines=True)
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [43]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

From the information above, we will be dealing with a lot more missing data. 
Let's take care of the easy transformations first, such as extracting the receipt_id and converting the dates

In [45]:
receipts['_id'] = [row['$oid'] for row in receipts['_id']]
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [46]:
receipts['createDate'] = receipts['createDate'].apply(lambda row: extract_date(row))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda row: extract_date(row))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda row: extract_date(row))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda row: extract_date(row))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda row: extract_date(row))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda row: extract_date(row))
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:36,2021-01-03 07:25:31,500.0,2021-01-02 16:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:48,2021-01-03 07:24:43,150.0,2021-01-02 07:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 07:25:37,2021-01-03 07:25:37,None,2021-01-03 07:25:42,None,5.0,2021-01-02 16:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:39,2021-01-03 07:25:34,5.0,2021-01-02 16:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 07:25:06,2021-01-03 07:25:06,2021-01-03 07:25:11,2021-01-03 07:25:11,2021-01-03 07:25:06,5.0,2021-01-02 07:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [47]:
receipts.rename(columns={'_id':'receipt_id'}, inplace=True)
receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:36,2021-01-03 07:25:31,500.0,2021-01-02 16:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:48,2021-01-03 07:24:43,150.0,2021-01-02 07:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 07:25:37,2021-01-03 07:25:37,None,2021-01-03 07:25:42,None,5.0,2021-01-02 16:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:39,2021-01-03 07:25:34,5.0,2021-01-02 16:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 07:25:06,2021-01-03 07:25:06,2021-01-03 07:25:11,2021-01-03 07:25:11,2021-01-03 07:25:06,5.0,2021-01-02 07:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


Let's start of by looking at the bonusPointsEarned and the bonusPointsEarnedReason columns

In [48]:
receipts['bonusPointsEarned'].unique()

array([500., 150.,   5., 750., 250., 100., 300.,  nan,  40.,  25.,  45.,
        27.,  21.])

In [51]:
receipts['bonusPointsEarnedReason'].unique()

array(['Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'All-receipts receipt bonus',
       'Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 3 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 6 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       'Receipt number 4 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
       nan, 'COMPLETE_PARTNER_RECEIPT', 'COMPLETE_NONPARTNER_RECEIPT'],
      dtype=object)

In [59]:
receipts.loc[receipts['bonusPointsEarned'].isna()]

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
15,5ff1e1e90a7214ada1000569,NaN,NaN,2021-01-03 07:25:29,2021-01-03 07:25:29,None,2021-01-03 07:25:29,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.00,5ff1e1e9b6a9d73a3a9f10f6
28,5ff1e1d40a7214ada1000562,NaN,NaN,2021-01-03 07:25:08,2021-01-03 07:25:08,None,2021-01-03 07:25:08,None,NaN,2017-10-29 17:00:00,3.0,"[{'deleted': True, 'description': 'DELETED ITE...",REJECTED,3.00,5ff1e194b6a9d73a3a9f1052
32,5ff36c750a7214ada100058f,NaN,NaN,2021-01-04 11:28:53,2021-01-04 11:28:53,2021-01-04 11:28:54,2021-01-04 11:28:54,2021-01-04 11:28:54,500.0,2021-01-03 11:28:53,9.0,"[{'barcode': '029000079236', 'description': 'P...",FINISHED,89.91,5ff36be7135e7011bcb856d3
35,5ff36adb0a720f0523000590,NaN,NaN,2021-01-04 11:22:03,2021-01-04 11:22:03,2021-01-04 11:22:04,2021-01-04 11:22:04,2021-01-04 11:22:04,250.0,2020-12-29 22:00:00,5.0,"[{'barcode': '044700009888', 'description': 'O...",FINISHED,49.95,5ff36a3862fde912123a4460
52,5ff3713c0a7214ada10005b6,NaN,NaN,2021-01-04 11:49:16,2021-01-04 11:49:16,2021-01-04 11:49:16,2021-01-04 11:49:16,2021-01-04 11:49:16,350.0,2021-01-03 11:49:16,1.0,"[{'barcode': '044700019917', 'description': 'O...",FINISHED,10.00,5ff370c562fde912123a5e0e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,603c6adf0a720fde1000039a,NaN,NaN,2021-02-28 20:17:35,2021-02-28 20:17:35,None,2021-02-28 20:17:35,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1111,603c9e6e0a720fde100003c7,NaN,NaN,2021-02-28 23:57:34,2021-02-28 23:57:34,None,2021-02-28 23:57:34,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 07:42:41,2021-03-01 07:42:41,None,2021-03-01 07:42:41,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 06:07:37,2021-03-01 06:07:37,None,2021-03-01 06:07:37,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33


In [60]:
# Direct examples for better visualization
receipts.iloc[1116]

receipt_id                 603cf5290a720fde10000413
bonusPointsEarned                               NaN
bonusPointsEarnedReason                         NaN
createDate                      2021-03-01 06:07:37
dateScanned                     2021-03-01 06:07:37
finishedDate                                   None
modifyDate                      2021-03-01 06:07:37
pointsAwardedDate                              None
pointsEarned                                    NaN
purchaseDate                                   None
purchasedItemCount                              NaN
rewardsReceiptItemList                          NaN
rewardsReceiptStatus                      SUBMITTED
totalSpent                                      NaN
userId                     5fc961c3b8cfca11a077dd33
Name: 1116, dtype: object

In [61]:
receipts.iloc[32]

receipt_id                                          5ff36c750a7214ada100058f
bonusPointsEarned                                                        NaN
bonusPointsEarnedReason                                                  NaN
createDate                                               2021-01-04 11:28:53
dateScanned                                              2021-01-04 11:28:53
finishedDate                                             2021-01-04 11:28:54
modifyDate                                               2021-01-04 11:28:54
pointsAwardedDate                                        2021-01-04 11:28:54
pointsEarned                                                           500.0
purchaseDate                                             2021-01-03 11:28:53
purchasedItemCount                                                       9.0
rewardsReceiptItemList     [{'barcode': '029000079236', 'description': 'P...
rewardsReceiptStatus                                                FINISHED

Observing when the missing data occurs and seeing how it occurs with relation to other coulmns is actually really helpful here. The big thing to notice is that missing data for the two columns we are looking at occurs simultaneously and is independent of whether points are awarded at all. Hence, I will be choosing to fill out the missing data in the bonusPointsEarned column with 0s and adding a new values to the reason column called 'No bonus points earned'. While we are again going into the territory of introducing bias into our data; however, after observing when the data is missing this seems to be the best step to make it more usable.

In [64]:
receipts['bonusPointsEarned'] = receipts['bonusPointsEarned'].replace({np.nan : 0})
receipts['bonusPointsEarnedReason'] = receipts['bonusPointsEarnedReason'].replace({np.nan : 'No bonus points awarded'})
receipts['bonusPointsEarned'] = receipts['bonusPointsEarned'].astype(int)
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   receipt_id               1119 non-null   object 
 1   bonusPointsEarned        1119 non-null   int32  
 2   bonusPointsEarnedReason  1119 non-null   object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

In [65]:
receipts.iloc[32]

receipt_id                                          5ff36c750a7214ada100058f
bonusPointsEarned                                                          0
bonusPointsEarnedReason                              No bonus points awarded
createDate                                               2021-01-04 11:28:53
dateScanned                                              2021-01-04 11:28:53
finishedDate                                             2021-01-04 11:28:54
modifyDate                                               2021-01-04 11:28:54
pointsAwardedDate                                        2021-01-04 11:28:54
pointsEarned                                                           500.0
purchaseDate                                             2021-01-03 11:28:53
purchasedItemCount                                                       9.0
rewardsReceiptItemList     [{'barcode': '029000079236', 'description': 'P...
rewardsReceiptStatus                                                FINISHED

Above, I made the changes that I suggested and also converted the bonusPointsEarned column to int instead of float to save space later on. Now that we have taken care of these two columns, let's move onto the next ones.

#### **Important:**
While the above changes make the data more interpretable, it is crucial to understand that we have introduced values into the data. This means that any aggregations performed on the data would need to factor that in now. For example, taking the average bonusPoints Awarded before would result in a much higher value as it is skipping the rows with missing data.

In [68]:
receipts.loc[receipts['finishedDate'].notna()]

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedu...",2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:36,2021-01-03 07:25:31,500.0,2021-01-02 16:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedu...",2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:48,2021-01-03 07:24:43,150.0,2021-01-02 07:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
3,5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:39,2021-01-03 07:25:34,5.0,2021-01-02 16:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5,All-receipts receipt bonus,2021-01-03 07:25:06,2021-01-03 07:25:06,2021-01-03 07:25:11,2021-01-03 07:25:11,2021-01-03 07:25:06,5.0,2021-01-02 07:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
5,5ff1e1e40a7214ada1000566,750,"Receipt number 1 completed, bonus point schedu...",2021-01-03 07:25:24,2021-01-03 07:25:24,2021-01-03 07:25:25,2021-01-03 07:25:30,2021-01-03 07:25:25,750.0,2021-01-02 07:25:24,1.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,3.25,5ff1e1e4cfcf6c399c274ac3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,60268cab0a7214d8e9000315,5,All-receipts receipt bonus,2021-02-12 06:11:55,2021-02-12 06:11:55,2021-02-12 06:11:58,2021-02-12 06:12:03,2021-02-12 06:11:57,5.0,2021-02-11 06:11:55,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,54943462e4b07e684157a532
984,60268caf0a720f05a80002e2,25,COMPLETE_NONPARTNER_RECEIPT,2021-02-12 06:11:59,2021-02-12 06:11:59,2021-02-12 06:12:00,2021-02-12 06:12:05,2021-02-12 06:12:00,35.0,2021-02-11 06:11:59,1.0,"[{'barcode': '079400066619', 'competitiveProdu...",FINISHED,1.00,60268c69efa6011bb151075f
985,60268c7e0a7214d8e9000309,100,"Receipt number 6 completed, bonus point schedu...",2021-02-12 06:11:10,2021-02-12 06:11:10,2021-02-12 06:11:11,2021-02-12 06:11:16,2021-02-12 06:11:11,100.0,2021-02-11 16:00:00,3.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,29.00,60268c7bb545931ac63683af
990,60268c790a7214d8e9000306,750,"Receipt number 1 completed, bonus point schedu...",2021-02-12 06:11:05,2021-02-12 06:11:05,2021-02-12 06:11:05,2021-02-12 06:11:11,2021-02-12 06:11:05,850.0,2021-02-11 16:00:00,1.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,25.00,60268c78efa6011bb151077d


With the finishedDate column, there is unfortunately not a lot that can be done. There does not seem to be a strong relation to any other columns. These receipts could still be processing, the data is lost, or a myriad of other reasons. Hence, I am choosing to leave it as is as the only other option would be to make it a set date or make it the same as modifyDate which in my opinion would lead to substantial issues with the quality of the data.

In [69]:
receipts.loc[receipts['pointsAwardedDate'].isna()]

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
2,5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,2021-01-03 07:25:37,2021-01-03 07:25:37,None,2021-01-03 07:25:42,None,5.0,2021-01-02 16:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
12,5ff1e1b60a7214ada100055c,150,"Receipt number 5 completed, bonus point schedu...",2021-01-03 07:24:38,2021-01-03 07:24:38,None,2021-01-03 07:24:38,None,8850.0,2021-02-03 07:24:38,10.0,"[{'barcode': '034100573065', 'description': 'M...",FLAGGED,290.00,5ff1e194b6a9d73a3a9f1052
15,5ff1e1e90a7214ada1000569,0,No bonus points awarded,2021-01-03 07:25:29,2021-01-03 07:25:29,None,2021-01-03 07:25:29,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.00,5ff1e1e9b6a9d73a3a9f10f6
28,5ff1e1d40a7214ada1000562,0,No bonus points awarded,2021-01-03 07:25:08,2021-01-03 07:25:08,None,2021-01-03 07:25:08,None,NaN,2017-10-29 17:00:00,3.0,"[{'deleted': True, 'description': 'DELETED ITE...",REJECTED,3.00,5ff1e194b6a9d73a3a9f1052
62,5ff4a4ca0a7214ada10005d0,750,"Receipt number 1 completed, bonus point schedu...",2021-01-05 09:41:30,2021-01-05 09:41:30,None,2021-01-05 09:41:32,None,750.0,2020-09-12 17:00:00,2.0,[{'description': 'mueller austria hypergrind p...,REJECTED,34.96,5fbc35711d967d1222cbfefc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 02:22:27,2021-03-01 02:22:27,None,2021-03-01 02:22:28,None,25.0,2020-08-16 17:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,0,No bonus points awarded,2021-03-01 07:42:41,2021-03-01 07:42:41,None,2021-03-01 07:42:41,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,0,No bonus points awarded,2021-03-01 06:07:37,2021-03-01 06:07:37,None,2021-03-01 06:07:37,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 05:07:28,2021-03-01 05:07:28,None,2021-03-01 05:07:29,None,25.0,2020-08-16 17:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [71]:
receipts.loc[receipts['pointsEarned'].isna()]

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
15,5ff1e1e90a7214ada1000569,0,No bonus points awarded,2021-01-03 07:25:29,2021-01-03 07:25:29,None,2021-01-03 07:25:29,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff1e1e9b6a9d73a3a9f10f6
28,5ff1e1d40a7214ada1000562,0,No bonus points awarded,2021-01-03 07:25:08,2021-01-03 07:25:08,None,2021-01-03 07:25:08,None,NaN,2017-10-29 17:00:00,3.0,"[{'deleted': True, 'description': 'DELETED ITE...",REJECTED,3.0,5ff1e194b6a9d73a3a9f1052
71,5ff475820a7214ada10005cf,0,No bonus points awarded,2021-01-05 06:19:46,2021-01-05 06:19:46,None,2021-01-05 06:19:46,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5a43c08fe4b014fd6b6a0612
81,5ff4ce3c0a720f05230005c4,0,No bonus points awarded,2021-01-05 12:38:20,2021-01-05 12:38:20,None,2021-01-05 12:38:20,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff4ce3cc1e2d0121a9b2fba
93,5ff5ecb90a7214ada10005f9,0,No bonus points awarded,2021-01-06 09:00:40,2021-01-06 09:00:40,None,2021-01-06 09:00:40,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5a43c08fe4b014fd6b6a0612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,603c6adf0a720fde1000039a,0,No bonus points awarded,2021-02-28 20:17:35,2021-02-28 20:17:35,None,2021-02-28 20:17:35,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1111,603c9e6e0a720fde100003c7,0,No bonus points awarded,2021-02-28 23:57:34,2021-02-28 23:57:34,None,2021-02-28 23:57:34,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,0,No bonus points awarded,2021-03-01 07:42:41,2021-03-01 07:42:41,None,2021-03-01 07:42:41,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,0,No bonus points awarded,2021-03-01 06:07:37,2021-03-01 06:07:37,None,2021-03-01 06:07:37,None,NaN,None,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33


After observing the information above, we can say that there is a much bigger issue at hand here. Points are being awarded, but the moment that they are is not being captured. One thing to consider would be using the modifiedDate to fill in the pointsAwardedDate; however, if you look at cases when it is not missing it would have to be the same as finishedDate. Hence, it is not possible to extrapolate any data and fill in the missing date values. However, for the points earned it is fine to put in 0s instead as there are no points being awarded in those situations.

In [81]:
receipts['pointsEarned'] = receipts['pointsEarned'].replace({np.nan : 0})
receipts['pointsEarned'] = receipts['pointsEarned'].astype(int)
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   receipt_id               1119 non-null   object 
 1   bonusPointsEarned        1119 non-null   int32  
 2   bonusPointsEarnedReason  1119 non-null   object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             1119 non-null   int32  
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

Moving forward, the purchasedDate column cannot be filled as there is no relation between when the items are purchased and when the receipt is scanned. 

In [91]:
receipts['totalSpent'] = receipts['totalSpent'].replace({np.nan : 0})
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   receipt_id               1119 non-null   object 
 1   bonusPointsEarned        1119 non-null   int32  
 2   bonusPointsEarnedReason  1119 non-null   object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             1119 non-null   int32  
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               1119 non-null   float64
 14  userId                  

After observing the totalSpent column, we notice that it is always missing data when there are no items purchased. Hence, I choose to make it 0 in those cases.

Finally, we look at purchasedItemCount and rewardsReceiptItemList.

In [100]:
ind = receipts.loc[receipts['purchasedItemCount'].isna() & receipts['rewardsReceiptItemList'].notna()].index
ind

Int64Index([210, 211, 213, 214, 215, 218, 219, 220, 221, 224, 225, 226, 227,
            228, 231, 232, 233, 235, 237, 238, 555, 759, 760, 761, 762, 763,
            764, 765, 766, 767, 768, 769, 770, 771, 772, 774, 775, 776, 777,
            778, 779, 780, 781, 783, 784, 785, 786, 787, 788],
           dtype='int64')

Here we can see that there are several instances when items have been purchased, but they have not translated to data being added to the item count. For these cases, we can traverse the data in the rewardsReceiptItemList and fill in the data, but otherwise we will fill it up with 0s.

In [126]:
receipts['purchasedItemCount'] = receipts['purchasedItemCount'].replace({np.nan : 0})
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   receipt_id               1119 non-null   object 
 1   bonusPointsEarned        1119 non-null   int32  
 2   bonusPointsEarnedReason  1119 non-null   object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             1119 non-null   int32  
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       1119 non-null   float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               1119 non-null   float64
 14  userId                  

In [127]:
receipts.at[ind, 'purchasedItemCount'] = 1
receipts.iloc[ind]

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
210,5ffe1cbe0a7214ad28002843,0,No bonus points awarded,2021-01-12 14:03:42,2021-01-12 14:03:42,2021-01-12 14:03:42,2021-01-12 14:03:42,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
211,5ffce7db0a720f051500236e,0,No bonus points awarded,2021-01-11 16:05:47,2021-01-11 16:05:47,2021-01-11 16:05:49,2021-01-11 16:05:47,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
213,5ffce8310a7214ad4e003797,0,No bonus points awarded,2021-01-11 16:07:13,2021-01-11 16:07:13,2021-01-11 16:07:14,2021-01-11 16:07:13,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
214,5ffe19d90a7214ad28000e62,0,No bonus points awarded,2021-01-12 13:51:21,2021-01-12 13:51:21,2021-01-12 13:51:21,2021-01-12 13:51:21,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
215,5ffe1d030a720f05ac002c9e,0,No bonus points awarded,2021-01-12 14:04:51,2021-01-12 14:04:51,2021-01-12 14:04:51,2021-01-12 14:04:51,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
218,5ffce76e0a720f0515000b48,0,No bonus points awarded,2021-01-11 16:03:58,2021-01-11 16:03:58,2021-01-11 16:04:00,2021-01-11 16:03:58,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
219,5ffe23560a720f05ac006874,0,No bonus points awarded,2021-01-12 14:31:50,2021-01-12 14:31:50,2021-01-12 14:31:50,2021-01-12 14:31:50,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
220,5ffe22a20a720f05ac0061d7,0,No bonus points awarded,2021-01-12 14:28:50,2021-01-12 14:28:50,2021-01-12 14:28:50,2021-01-12 14:28:50,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
221,5ffe1dc20a7214ad28003180,0,No bonus points awarded,2021-01-12 14:08:02,2021-01-12 14:08:02,2021-01-12 14:08:03,2021-01-12 14:08:02,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330
224,5ffe1aa90a7214ad280015e8,0,No bonus points awarded,2021-01-12 13:54:49,2021-01-12 13:54:49,2021-01-12 13:54:49,2021-01-12 13:54:49,None,0,2021-01-11 16:00:00,1.0,[{'description': 'flipbelt level terrain waist...,PENDING,28.57,59c124bae4b0299e55b0f330


In [244]:
receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedu...",2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:36,2021-01-03 07:25:31,500,2021-01-02 16:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedu...",2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:48,2021-01-03 07:24:43,150,2021-01-02 07:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,2021-01-03 07:25:37,2021-01-03 07:25:37,None,2021-01-03 07:25:42,None,5,2021-01-02 16:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:39,2021-01-03 07:25:34,5,2021-01-02 16:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5,All-receipts receipt bonus,2021-01-03 07:25:06,2021-01-03 07:25:06,2021-01-03 07:25:11,2021-01-03 07:25:11,2021-01-03 07:25:06,5,2021-01-02 07:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


Now we have fixed the issue where items were purchased, but not counted. An interesting thing to note, this only occured for the item 'flipbelt level terrain waist pouch, neon yellow, large/32-35' and a singular instance of a Taco Bell receipt.

In [128]:
receipts['rewardsReceiptItemList'][0]

[{'barcode': '4011',
  'description': 'ITEM NOT FOUND',
  'finalPrice': '26.00',
  'itemPrice': '26.00',
  'needsFetchReview': False,
  'partnerItemId': '1',
  'preventTargetGapPoints': True,
  'quantityPurchased': 5,
  'userFlaggedBarcode': '4011',
  'userFlaggedNewItem': True,
  'userFlaggedPrice': '26.00',
  'userFlaggedQuantity': 5}]

Since each instance in the rewardsReceiptItemList column seems to have a bunch of features in it, I will explode the column to get each item in its own row. Additionally, I will be doing this in order to create a new table called receipt_items in order to not overly clutter the receipts table.

In [206]:
receipt_items = receipts[['receipt_id', 'rewardsReceiptItemList']].copy()
receipt_items.head()

,receipt_id,rewardsReceiptItemList
0,5ff1e1eb0a720f0523000575,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
1,5ff1e1bb0a720f052300056b,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
2,5ff1e1f10a720f052300057a,"[{'needsFetchReview': False, 'partnerItemId': ..."
3,5ff1e1ee0a7214ada100056f,"[{'barcode': '4011', 'description': 'ITEM NOT ..."
4,5ff1e1d20a7214ada1000561,"[{'barcode': '4011', 'description': 'ITEM NOT ..."


In [207]:
receipt_items.shape

(1119, 2)

In [208]:
receipt_items = receipt_items.explode('rewardsReceiptItemList')
receipt_items.head()

,receipt_id,rewardsReceiptItemList
0,5ff1e1eb0a720f0523000575,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,5ff1e1bb0a720f052300056b,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,5ff1e1bb0a720f052300056b,"{'barcode': '028400642255', 'description': 'DO..."
2,5ff1e1f10a720f052300057a,"{'needsFetchReview': False, 'partnerItemId': '..."
3,5ff1e1ee0a7214ada100056f,"{'barcode': '4011', 'description': 'ITEM NOT F..."


In [209]:
receipt_items.reset_index(inplace=True)

In [210]:
receipt_items.head()

,index,receipt_id,rewardsReceiptItemList
0,0,5ff1e1eb0a720f0523000575,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,1,5ff1e1bb0a720f052300056b,"{'barcode': '4011', 'description': 'ITEM NOT F..."
2,1,5ff1e1bb0a720f052300056b,"{'barcode': '028400642255', 'description': 'DO..."
3,2,5ff1e1f10a720f052300057a,"{'needsFetchReview': False, 'partnerItemId': '..."
4,3,5ff1e1ee0a7214ada100056f,"{'barcode': '4011', 'description': 'ITEM NOT F..."


In [211]:
receipt_items.shape

(7381, 3)

After exploding the column, we can see that there were several rows that included multiple items in them. Additionally, we now have an index column as well that helps group items from the same receipt (index column can be dropped as receipt_id is enough to see grouping).

In [212]:
receipt_items['rewardsReceiptItemList'] = receipt_items['rewardsReceiptItemList'].replace({np.nan : '{}'})
receipt_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   7381 non-null   int64 
 1   receipt_id              7381 non-null   object
 2   rewardsReceiptItemList  7381 non-null   object
dtypes: int64(1), object(2)
memory usage: 173.1+ KB


In [213]:
receipt_items.head()

,index,receipt_id,rewardsReceiptItemList
0,0,5ff1e1eb0a720f0523000575,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,1,5ff1e1bb0a720f052300056b,"{'barcode': '4011', 'description': 'ITEM NOT F..."
2,1,5ff1e1bb0a720f052300056b,"{'barcode': '028400642255', 'description': 'DO..."
3,2,5ff1e1f10a720f052300057a,"{'needsFetchReview': False, 'partnerItemId': '..."
4,3,5ff1e1ee0a7214ada100056f,"{'barcode': '4011', 'description': 'ITEM NOT F..."


In [214]:
receipt_items['rewardsReceiptItemList'] = receipt_items['rewardsReceiptItemList'].apply(lambda x: str(x))
receipt_items['rewardsReceiptItemList'] = receipt_items['rewardsReceiptItemList'].apply(ast.literal_eval)
receipt_items.head()

,index,receipt_id,rewardsReceiptItemList
0,0,5ff1e1eb0a720f0523000575,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,1,5ff1e1bb0a720f052300056b,"{'barcode': '4011', 'description': 'ITEM NOT F..."
2,1,5ff1e1bb0a720f052300056b,"{'barcode': '028400642255', 'description': 'DO..."
3,2,5ff1e1f10a720f052300057a,"{'needsFetchReview': False, 'partnerItemId': '..."
4,3,5ff1e1ee0a7214ada100056f,"{'barcode': '4011', 'description': 'ITEM NOT F..."


In [215]:
temp = pd.json_normalize(receipt_items['rewardsReceiptItemList'], errors = 'ignore')
receipt_items = pd.merge(receipt_items, temp, how = 'outer', left_index = True, right_index = True)
receipt_items.head()

,index,receipt_id,rewardsReceiptItemList,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,"{'barcode': '4011', 'description': 'ITEM NOT F...",4011,ITEM NOT FOUND,26.00,26.00,False,1,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,"{'barcode': '4011', 'description': 'ITEM NOT F...",4011,ITEM NOT FOUND,1,1,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,"{'barcode': '028400642255', 'description': 'DO...",028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,"{'needsFetchReview': False, 'partnerItemId': '...",NaN,NaN,NaN,NaN,False,1,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,"{'barcode': '4011', 'description': 'ITEM NOT F...",4011,ITEM NOT FOUND,28.00,28.00,False,1,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
#receipt_items.drop(columns=['rewardsReceiptItemList'], inplace=True)
receipt_items.head()

,index,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
receipt_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7381 entries, 0 to 7380
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   index                               7381 non-null   int64  
 1   receipt_id                          7381 non-null   object 
 2   barcode                             3090 non-null   object 
 3   description                         6560 non-null   object 
 4   finalPrice                          6767 non-null   object 
 5   itemPrice                           6767 non-null   object 
 6   needsFetchReview                    813 non-null    object 
 7   partnerItemId                       6941 non-null   object 
 8   preventTargetGapPoints              358 non-null    object 
 9   quantityPurchased                   6767 non-null   float64
 10  userFlaggedBarcode                  337 non-null    object 
 11  userFlaggedNewItem                  323 non

Let's recap everyhting that has happened so far. The column rewardsReceiptItemList contained a lot of information that could be useful, as you can see above most of the data is missing but there are some instances where it is still there. The best way to tackle this would be to drop all columns that have excessive amounts of data missing from them. Most columns here cannot be filled with a set value or extrapolated from values present in other columns so this is a step that would have to be taken.

In [232]:
ind = receipt_items.loc[receipt_items['barcode'].isna() & receipt_items['description'].isna()].index
receipt_items = receipt_items.drop(ind)

In [234]:
receipt_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6791 entries, 0 to 7379
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   index                               6791 non-null   int64  
 1   receipt_id                          6791 non-null   object 
 2   barcode                             3090 non-null   object 
 3   description                         6560 non-null   object 
 4   finalPrice                          6767 non-null   object 
 5   itemPrice                           6767 non-null   object 
 6   needsFetchReview                    663 non-null    object 
 7   partnerItemId                       6791 non-null   object 
 8   preventTargetGapPoints              208 non-null    object 
 9   quantityPurchased                   6767 non-null   float64
 10  userFlaggedBarcode                  187 non-null    object 
 11  userFlaggedNewItem                  173 non

What we did above was to drop rows where there is no barcode present. These rows cannot provide us with any information so we get rid of them. Additionally, we ensured that we kept the rows where there is an item description since these rows can still provide us with value. 

In [241]:
cols = [8, 10, 11, 12, 13, 14, 15, 19, 20, 21, 26, 27, 31, 32, 33]
receipt_items.drop(receipt_items.columns[cols], inplace=True, axis=1)
receipt_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6791 entries, 0 to 7379
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    6791 non-null   int64  
 1   receipt_id               6791 non-null   object 
 2   barcode                  3090 non-null   object 
 3   description              6560 non-null   object 
 4   finalPrice               6767 non-null   object 
 5   itemPrice                6767 non-null   object 
 6   needsFetchReview         663 non-null    object 
 7   partnerItemId            6791 non-null   object 
 8   quantityPurchased        6767 non-null   float64
 9   pointsPayerId            1267 non-null   object 
 10  rewardsGroup             1731 non-null   object 
 11  rewardsProductPartnerId  2269 non-null   object 
 12  brandCode                2600 non-null   object 
 13  competitorRewardsGroup   275 non-null    object 
 14  discountedItemPrice     

Above we removed all the columns that had over 97% of the data missing. In a work environment, the columns to drop would require some deliberation between the team in order to decide what we need for our solution, but in either case it would be imperative to discover what is causing the lack of data for several of these features. 

#### Overview
Now that I'm done preparing the data to be placed in a warehouse, let's look back at the last two tables we worked with. The receipts table had a lot of missing data and we worked around several of the issues that we were facing. When it came to the itemList column, I decided it was better to create a new table and thus receipt_items was made. In this table, we were faced with even more missing data and several columns were dropped for reasons explained above. In terms of optimizing the last table for storage and feature engineering, it would depend heavily on what we want our product to do. As things stand, the four tables that we have should be able to answer most if not all questions that can be asked. However, the first thing that I would do would be clarify what is required and what isn't from the last table and clean it up more and then study where we are facing data leakage as several of the columns should not be missing data based on what else is collected.

## Step 2: Answer Stakeholder Questions

In [247]:
import sqlalchemy as sqla

In [248]:
engine = sqla.create_engine('sqlite://', echo=False)

In [258]:
sql = 'DROP TABLE IF EXISTS receipts;'
result = engine.execute(sql)

In [259]:
#receipts.drop(columns=['rewardsReceiptItemList'], axis=1, inplace=True)
receipts.drop_duplicates(subset=['receipt_id']).to_sql('receipts',con=engine)

C:\Users\Akshat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [268]:
receipts.columns

Index(['receipt_id', 'bonusPointsEarned', 'bonusPointsEarnedReason',
       'createDate', 'dateScanned', 'finishedDate', 'modifyDate',
       'pointsAwardedDate', 'pointsEarned', 'purchaseDate',
       'purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent', 'userId'],
      dtype='object')

In [272]:
receipts.head()

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedu...",2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:31,2021-01-03 07:25:36,2021-01-03 07:25:31,500,2021-01-02 16:00:00,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedu...",2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:43,2021-01-03 07:24:48,2021-01-03 07:24:43,150,2021-01-02 07:24:43,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,2021-01-03 07:25:37,2021-01-03 07:25:37,None,2021-01-03 07:25:42,None,5,2021-01-02 16:00:00,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:34,2021-01-03 07:25:39,2021-01-03 07:25:34,5,2021-01-02 16:00:00,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5,All-receipts receipt bonus,2021-01-03 07:25:06,2021-01-03 07:25:06,2021-01-03 07:25:11,2021-01-03 07:25:11,2021-01-03 07:25:06,5,2021-01-02 07:25:06,2.0,FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [273]:
receipts['rewardsReceiptStatus'].unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [287]:
sql = "Select AVG(totalSpent) From receipts Where rewardsReceiptStatus = 'REJECTED' AND totalSpent <> 0; "
engine.execute(sql).first()

(24.355147058823544,)

In [288]:
sql = "Select AVG(totalSpent) From receipts Where rewardsReceiptStatus = 'FINISHED' AND totalSpent <> 0; "
engine.execute(sql).first()

(81.1676937984496,)

From the above queries, we can see that when the rewardsReceiptStatus was rejected the average spending was **24.36** dollars. Whereas, when it was accepted it was **81.17** dollars. One thing to consider in the queries is that we are avoiding rows where the total spending is 0 since we put those values in. While including the rows does not cause a massive change to our values (~$1 for each), it is important to understand that if the data was different then this is something that would have to be considered (especially when choosing to replace the empty rows with 0).

## Step 3: Data Quality Issues

While we have have identified several data quality issues along the way, it never hurts to look for more. One thing that comes to mind is that there were several 'Test Brands' in the brand tables. With how many test brands there were, are they equally available in the actual transactions?

In [282]:
brands['brandCode'].nunique()

897

In [279]:
receipt_items['brandCode'].nunique()

227

Looking above, we notice that there are 897 unique brands that we have data about. However, when it comes to actual transactions there are only 227. While some could say that some of the brands are just being bought a lot more, but we must consider that there was a lot of data missing in the transaction data. This could have led to several brands not being represented properly which could paint a different picture than what is actually happening. 

While I discovered another potential issue, I would like to bring attention back to what I think the main issue is: points being awarded without data about the process being logged. This is more likely to be an error within our own system which is more avoidable and more concerning from a data standpoint as it could be a sign of issues with our data collection process.